### [How to stream tool calls](https://python.langchain.com/docs/how_to/tool_streaming/)

In [1]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Model

In [2]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

In [4]:
query = "What is 3 * 12? Also, what is 11 + 49?"

async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '', 'id': 'call_GGiL6SYnIAsBo0wxV7GvdQtb', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'add', 'args': '', 'id': 'call_cAdl1lOvsAWWUv1gZNRfJUq3', 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[]


In [5]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '', 'id': 'call_BfZqQkIoEQxJlOB3gb2yz4Lv', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a"', 'id': 'call_BfZqQkIoEQxJlOB3gb2yz4Lv', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, ', 'id': 'call_BfZqQkIoEQxJlOB3gb2yz4Lv', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, "b": 1', 'id': 'call_BfZqQkIoEQxJlOB3gb2yz4Lv', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_BfZqQkIoEQxJlOB3gb2yz4Lv', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_BfZqQkIoEQxJlOB3gb2yz4Lv', 'index': 0, 'type': 'tool_call_chunk'}, {'name': 'add', 'args': '', 'id': 'call_8YDI3Oyk9Y0ccwsZr8vvTWNn', 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_BfZqQkIoEQxJlOB3gb2yz4Lv', 'index': 0, 'type': 'tool_call_chunk'}, {'name': 'add', 'arg

In [6]:
print(type(gathered.tool_call_chunks[0]["args"]))

<class 'str'>


In [7]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

[]
[{'name': 'multiply', 'args': {}, 'id': 'call_qd1d3a2AOc1A6AqDJyEHfo3P', 'type': 'tool_call'}]
[{'name': 'multiply', 'args': {}, 'id': 'call_qd1d3a2AOc1A6AqDJyEHfo3P', 'type': 'tool_call'}]
[{'name': 'multiply', 'args': {'a': 3}, 'id': 'call_qd1d3a2AOc1A6AqDJyEHfo3P', 'type': 'tool_call'}]
[{'name': 'multiply', 'args': {'a': 3, 'b': 1}, 'id': 'call_qd1d3a2AOc1A6AqDJyEHfo3P', 'type': 'tool_call'}]
[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_qd1d3a2AOc1A6AqDJyEHfo3P', 'type': 'tool_call'}]
[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_qd1d3a2AOc1A6AqDJyEHfo3P', 'type': 'tool_call'}, {'name': 'add', 'args': {}, 'id': 'call_bNvqeLfo9pf90BZwvGAGG5jl', 'type': 'tool_call'}]
[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_qd1d3a2AOc1A6AqDJyEHfo3P', 'type': 'tool_call'}, {'name': 'add', 'args': {}, 'id': 'call_bNvqeLfo9pf90BZwvGAGG5jl', 'type': 'tool_call'}]
[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_qd1d3a2AOc1A6AqDJyEHfo3P', '

In [8]:
print(type(gathered.tool_calls[0]["args"]))

<class 'dict'>
